In [2]:
from dotenv import load_dotenv

# 토큰 정보 로드
load_dotenv()

True

In [4]:
import pandas as pd
from sqlalchemy import create_engine

df= pd.read_csv("dumy.csv", encoding='utf-8')
engine = create_engine("sqlite:///employee.db")
df.to_sql("employee", engine, index=False)


5000

In [5]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from sqlalchemy import text
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate


db = SQLDatabase.from_uri("sqlite:///employee.db")
print(db.get_usable_table_names())

['employee']


In [30]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
sql = chain.invoke({"question": "회사에서 영업 부서에 속한 직원들은 몇명이야?"})
sql

'SELECT COUNT("Name") \nFROM employee \nWHERE "Department" = \'Sales\''

In [31]:
response = db.run(sql)
print(response)

[(1004,)]


In [32]:
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query

In [33]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
)

answer_chain=answer_prompt|llm
question="회사에서 영업 부서에 속한 직원들은 몇명이야?"
answer=answer_chain.invoke({"question": question, "query": sql, "result": response})
print(answer.content)

회사에서 영업 부서에 속한 직원은 1004명입니다.


In [34]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
agent_executor.invoke({"input": "회사에서 영업 부서에 속한 직원들은 몇명이야?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


employee
Invoking: `sql_db_schema` with `{'table_names': 'employee'}`



CREATE TABLE employee (
	"Unnamed: 0" BIGINT, 
	"Name" TEXT, 
	"Address" TEXT, 
	"Salary" FLOAT, 
	"DOJ" TEXT, 
	"DOB" TEXT, 
	"Age" BIGINT, 
	"Sex" TEXT, 
	"Dependents" FLOAT, 
	"HRA" FLOAT, 
	"DA" FLOAT, 
	"PF" FLOAT, 
	"Gross Salary" FLOAT, 
	"Insurance" TEXT, 
	"Marital Status" TEXT, 
	"In Company Years" BIGINT, 
	"Year of Experience" BIGINT, 
	"Department" TEXT, 
	"Position" TEXT
)

/*
3 rows from employee table:
Unnamed: 0	Name	Address	Salary	DOJ	DOB	Age	Sex	Dependents	HRA	DA	PF	Gross Salary	Insurance	Marital Status	In Company Years	Year of Experience	Department	Position
0	VgwbPyBn	jyNxKZtk St, piZQhYcARo, PUN 172813	149289.04	1998-04-27	1971-03-27	53	Other	2.0	8776.095999999998	35233.66637469586	22142.724764963503	171156.07760973237	None	Widowed	26	32	Sales	Sales Director
1	pgdYaIWk	PypVsqug St, iHyelKZbFc, BLR 580530	6

{'input': '회사에서 영업 부서에 속한 직원들은 몇명이야?', 'output': '영업 부서에 속한 직원은 1004명입니다.'}